# Summary
1. We select the built class of Google Dynamic World (GDW) and filter the certainty to be greater than 0.06, as the highest false positive in the grassland had a certainty of 0.0569.
2. We also mask resulting GDW layer with a rock mask, which we manually drew from false positives from the Google Dynamic World built class.
3. We import Google Open Buildings, 2022 (GOB), convert it to raster and combine the GDW.
4. Splitting up Africa in blocks, we then export the resulting combined built layer to Google Cloud Storage.

### Future
- Conduct similar analysis for all years
- Improve years before GDW (2022)

# Setting up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

In [ ]:
block_list = list(range(1, 26))

blocks = []
with open("/content/drive/MyDrive/data/blocks.geojson") as f:
    json_data = json.load(f)
    for block_id in block_list:
        for feature in json_data['features']:
            if feature['properties']['id'] == block_id:
                feature['properties']['style'] = {}
                blocks.append(feature)

ee_blocks = [geemap.geojson_to_ee(block) for block in blocks]

In [ ]:
# Import rockMask from GoogleCloud
fs = gcsfs.GCSFileSystem(project='nature-watch-387210')

with fs.open('gs://nature-watch-bucket/vector/rocks.geojson', 'r') as f:
    data = json.load(f)

rocks = geemap.geojson_to_ee(data)

In [ ]:
# Prepare layers
year = 2022
start_date = '{}-01-01'.format(year)
end_date = '{}-01-01'.format(year + 1)

# Google Dynamic World
people = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('label').eq(6).selfMask()
certainty_mask = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date, end_date).median().select('built').gt(0.06).selfMask()
best_people = people.mask(certainty_mask).eq(1).selfMask().unmask()

# Use the rock mask
raster_mask = ee.Image().paint(rocks, 1).Not()
best_people2 = best_people.updateMask(raster_mask)

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(rocks, {}, 'rocks')
Map.addLayer(best_people, {}, 'best_people')
Map.addLayer(best_people2, {}, 'best_people2')
Map.setCenter(31.944, -25.087, 12)
Map